### Create DataFrame

In [1]:
import numpy as np
import pandas as pd
import os

def make_catndog_dataframe():
    
    paths = []
    dataset_gubuns = []
    label_gubuns = []
    
    for dirname, _, filenames in os.walk('/kaggle/input/cat-and-dog'):
        for filename in filenames:
            if '.jpg' in filename:
                file_path = dirname + '/' + filename
                paths.append(file_path)
                if '/training_set/' in file_path:
                    dataset_gubuns.append('train')
                elif '/test_set/' in file_path:
                    dataset_gubuns.append('test')
                else:
                    dataset_gubuns.append('N/A')
                
                if 'dogs' in file_path:
                    label_gubuns.append('DOG')
                elif 'cats' in file_path:
                    label_gubuns.append('CAT')
                else:
                    label_gubuns.append('N/A')
    
    data_df = pd.DataFrame({
        'path': paths,
        'dataset': dataset_gubuns,
        'label': label_gubuns
    })
    
    return data_df

In [2]:
pd.set_option('display.max_colwidth', 200)
data_df = make_catndog_dataframe()
print('date_df shape: ', data_df.shape)
data_df.head()

date_df shape:  (10028, 3)


,path,dataset,label
0,/kaggle/input/cat-and-dog/test_set/test_set/dogs/dog.4329.jpg,test,DOG
1,/kaggle/input/cat-and-dog/test_set/test_set/dogs/dog.4223.jpg,test,DOG
2,/kaggle/input/cat-and-dog/test_set/test_set/dogs/dog.4253.jpg,test,DOG
3,/kaggle/input/cat-and-dog/test_set/test_set/dogs/dog.4190.jpg,test,DOG
4,/kaggle/input/cat-and-dog/test_set/test_set/dogs/dog.4354.jpg,test,DOG


### Create Sequence Dataset

In [3]:
from tensorflow.keras.utils import Sequence
import sklearn
import cv2

BATCH_SIZE = 64
IMAGE_SIZE = 160

class CnD_Dataset(Sequence):
    def __init__(self, image_filenames, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=None, pre_func=None):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        
        self.shuffle = shuffle
        
        if self.shuffle:
            pass
        
    def __len__(self):
        return int(np.ceil(len(self.image_filenames)/BATCH_SIZE))
    
    def __getitem__(self, index):
        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]
        
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            
        image_batch = np.zeros((image_name_batch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
        for image_index in range(image_name_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    def on_eopch_end(self):
        if (self.shuffle):
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames, self.labels)
        else:
            pass

### Dataset Split on Train/Validation/Test

In [4]:
from sklearn.model_selection import train_test_split

def get_train_valid_test(data_df):
    train_df = data_df[data_df['dataset']=='train']
    test_df = data_df[data_df['dataset']=='test']
    
    train_path = train_df['path'].values
    train_label = pd.factorize(train_df['label'])[0]
    
    test_path = test_df['path'].values
    test_label = pd.factorize(test_df['label'])[0]
    
    tr_path, val_path, tr_label, val_label = train_test_split(train_path, train_label, test_size=0.5, random_state=2021)
    
    print('학습용 path shape:', tr_path.shape, '검증용 path shape:', val_path.shape, 
      '학습용 label shape:', tr_label.shape, '검증용 label shape:', val_label.shape)
    return tr_path, tr_label, val_path, val_label, test_path, test_label

### Create Pretrained Model

In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import Xception, MobileNetV2

def create_model(model_name='mobilenet', verbose=False):
    
    input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    if model_name == 'vgg16':
        base_model = VGG16(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'resnet50':
        base_model = ResNet50V2(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'xception':
        base_model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'mobilenet':
        base_model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')
    
    bm_output = base_model.output

    x = GlobalAveragePooling2D()(bm_output)
    if model_name != 'vgg16':
        x = Dropout(rate=0.5)(x)
    x = Dense(50, activation='relu', name='fc1')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_tensor, outputs=output)
    
    if verbose:
        model.summary()
        
    return model

### Train Pretrained Model

In [6]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_preprocess_input
   
def train_model(data_df, model_name, augmentor, preprocessing_func):

    tr_path, tr_label, val_path, val_label, test_path, test_label = get_train_valid_test(data_df)
    
    tr_ds = CnD_Dataset(tr_path, tr_label, batch_size=BATCH_SIZE, augmentor=augmentor, 
                          shuffle=True, pre_func=preprocessing_func)
    val_ds = CnD_Dataset(val_path, val_label, batch_size=BATCH_SIZE, augmentor=None, 
                           shuffle=False, pre_func=preprocessing_func)
    
    model = create_model(model_name=model_name)
    model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    N_EPOCHS = 20

    history = model.fit(tr_ds, epochs=N_EPOCHS, steps_per_epoch=int(np.ceil(tr_path.shape[0]/BATCH_SIZE)), 
                       validation_data=val_ds, validation_steps=int(np.ceil(val_path.shape[0]/BATCH_SIZE)),
                        verbose=1)
    
    return model, history

### MobileNetV2 Train and Evaluation

In [7]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_preprocess_input

input_df, _ = train_test_split(data_df, test_size=0.7, random_state=2021)

mobile_model, mobile_history = train_model(input_df, 'mobilenet', None, mobile_preprocess_input)

학습용 path shape: (1202,) 검증용 path shape: (1202,) 학습용 label shape: (1202,) 검증용 label shape: (1202,)
9412608/9406464 [==============================] - 0s 0us/step
Epoch 1/20
19/19 [==============================] - 30s 1s/step - loss: 0.6489 - accuracy: 0.6537 - val_loss: 0.0940 - val_accuracy: 0.9659
Epoch 2/20
19/19 [==============================] - 9s 507ms/step - loss: 0.1435 - accuracy: 0.9455 - val_loss: 0.0803 - val_accuracy: 0.9667
Epoch 3/20
19/19 [==============================] - 9s 487ms/step - loss: 0.0586 - accuracy: 0.9922 - val_loss: 0.0906 - val_accuracy: 0.9651
Epoch 4/20
19/19 [==============================] - 9s 491ms/step - loss: 0.0252 - accuracy: 1.0000 - val_loss: 0.0966 - val_accuracy: 0.9684
Epoch 5/20
19/19 [==============================] - 9s 489ms/step - loss: 0.0153 - accuracy: 0.9986 - val_loss: 0.1027 - val_accuracy: 0.9667
Epoch 6/20
19/19 [==============================] - 10s 526ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.1078 - val_accur

In [8]:
test_df = data_df[data_df['dataset']=='test']

test_path = test_df['path'].values
test_label = pd.factorize(test_df['label'])[0]

test_ds = CnD_Dataset(test_path, test_label, batch_size=BATCH_SIZE, augmentor=None,
                     shuffle=False, pre_func=mobile_preprocess_input)

mobile_model.evaluate(test_ds)

32/32 [==============================] - 16s 493ms/step - loss: 0.0856 - accuracy: 0.9787


[0.08557981997728348, 0.9787444472312927]

### Pretrained Model Layer Structure

In [9]:
model = create_model(model_name='mobilenet')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [10]:
print(type(model.layers))

<class 'list'>


In [12]:
model.layers

In [13]:
model.layers[-4:]

In [14]:
for layer in model.layers:
    print(layer.name, 'trainable: ', layer.trainable)

input_2 trainable:  True
Conv1 trainable:  True
bn_Conv1 trainable:  True
Conv1_relu trainable:  True
expanded_conv_depthwise trainable:  True
expanded_conv_depthwise_BN trainable:  True
expanded_conv_depthwise_relu trainable:  True
expanded_conv_project trainable:  True
expanded_conv_project_BN trainable:  True
block_1_expand trainable:  True
block_1_expand_BN trainable:  True
block_1_expand_relu trainable:  True
block_1_pad trainable:  True
block_1_depthwise trainable:  True
block_1_depthwise_BN trainable:  True
block_1_depthwise_relu trainable:  True
block_1_project trainable:  True
block_1_project_BN trainable:  True
block_2_expand trainable:  True
block_2_expand_BN trainable:  True
block_2_expand_relu trainable:  True
block_2_depthwise trainable:  True
block_2_depthwise_BN trainable:  True
block_2_depthwise_relu trainable:  True
block_2_project trainable:  True
block_2_project_BN trainable:  True
block_2_add trainable:  True
block_3_expand trainable:  True
block_3_expand_BN traina

In [15]:
for layer in model.layers[:-4]:
    layer.trainable = False
    print(layer.name, 'trainable: ', layer.trainable)

print('\n### final 4 layers ### ')

for layer in model.layers[-4:]:
    print(layer.name, 'trainable: ', layer.trainable)

input_2 trainable:  False
Conv1 trainable:  False
bn_Conv1 trainable:  False
Conv1_relu trainable:  False
expanded_conv_depthwise trainable:  False
expanded_conv_depthwise_BN trainable:  False
expanded_conv_depthwise_relu trainable:  False
expanded_conv_project trainable:  False
expanded_conv_project_BN trainable:  False
block_1_expand trainable:  False
block_1_expand_BN trainable:  False
block_1_expand_relu trainable:  False
block_1_pad trainable:  False
block_1_depthwise trainable:  False
block_1_depthwise_BN trainable:  False
block_1_depthwise_relu trainable:  False
block_1_project trainable:  False
block_1_project_BN trainable:  False
block_2_expand trainable:  False
block_2_expand_BN trainable:  False
block_2_expand_relu trainable:  False
block_2_depthwise trainable:  False
block_2_depthwise_BN trainable:  False
block_2_depthwise_relu trainable:  False
block_2_project trainable:  False
block_2_project_BN trainable:  False
block_2_add trainable:  False
block_3_expand trainable:  Fa

In [16]:
from tensorflow.keras import layers

def train_model_fine_tune(data_df, model_name, augmentor, preprocessing_func):
    tr_path, tr_label, val_path, val_label, test_path, test_label = get_train_valid_test(data_df)
    
    tr_ds = CnD_Dataset(tr_path, tr_label, batch_size=BATCH_SIZE, augmentor=augmentor, 
                          shuffle=True, pre_func=preprocessing_func)
    val_ds = CnD_Dataset(val_path, val_label, batch_size=BATCH_SIZE, augmentor=None, 
                           shuffle=False, pre_func=preprocessing_func)
    
    model = create_model(model_name=model_name)
    model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    
    for layer in model.layers[:-4]:
        layer.trainable = False
    
    FIRST_EPOCHS = 10
    SECOND_EPOCHS = 10

    history = model.fit(tr_ds, epochs=FIRST_EPOCHS, steps_per_epoch=int(np.ceil(tr_path.shape[0]/BATCH_SIZE)), 
                       validation_data=val_ds, validation_steps=int(np.ceil(val_path.shape[0]/BATCH_SIZE)),
                       verbose=1)

    for layer in model.layers:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    model.compile(optimizer=Adam(0.00001), loss='binary_crossentropy', metrics=['accuracy'])    
    history = model.fit(tr_ds, epochs=SECOND_EPOCHS, steps_per_epoch=int(np.ceil(tr_path.shape[0]/BATCH_SIZE)), 
                       validation_data=val_ds, validation_steps=int(np.ceil(val_path.shape[0]/BATCH_SIZE)),
                       verbose=1)
    
    return model, history

In [17]:
mobile_model_tuned, mobile_tuned_history = train_model_fine_tune(input_df, 'mobilenet', None, mobile_preprocess_input)

학습용 path shape: (1202,) 검증용 path shape: (1202,) 학습용 label shape: (1202,) 검증용 label shape: (1202,)
Epoch 1/10
19/19 [==============================] - 15s 561ms/step - loss: 0.6150 - accuracy: 0.6603 - val_loss: 0.1008 - val_accuracy: 0.9609
Epoch 2/10
19/19 [==============================] - 10s 527ms/step - loss: 0.1434 - accuracy: 0.9576 - val_loss: 0.0885 - val_accuracy: 0.9659
Epoch 3/10
19/19 [==============================] - 9s 491ms/step - loss: 0.0573 - accuracy: 0.9904 - val_loss: 0.1085 - val_accuracy: 0.9676
Epoch 4/10
19/19 [==============================] - 9s 483ms/step - loss: 0.0241 - accuracy: 0.9990 - val_loss: 0.1219 - val_accuracy: 0.9667
Epoch 5/10
19/19 [==============================] - 9s 491ms/step - loss: 0.0138 - accuracy: 1.0000 - val_loss: 0.1316 - val_accuracy: 0.9659
Epoch 6/10
19/19 [==============================] - 10s 534ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.1364 - val_accuracy: 0.9651
Epoch 7/10
19/19 [=============================

In [19]:
mobile_model_tuned.evaluate(test_ds)

32/32 [==============================] - 7s 220ms/step - loss: 0.0834 - accuracy: 0.9822


[0.08335588127374649, 0.9822046756744385]